In [1]:
import pandas as pd

import pandas as pd

# df1 = dataframe of manually encoded datasets.

df1 = pd.read_csv('dataset-december.csv', encoding='utf-8')
df1 = df1.drop(columns = ['id'])
df1

,question,label
0,It is the sum of the terms of a sequence.,series_and_sequence
1,Find the sum of the first ten terms of the ari...,series_and_sequence
2,Find the sum of the first 25 terms of the arit...,series_and_sequence
3,The sum of the first 10 terms of an arithmetic...,series_and_sequence
4,The third term of an arithmetic sequence is −1...,series_and_sequence
...,...,...
840,Determine the zeros of the polynomial function...,polynomial_equation
841,Determine the zeros of the polynomial function...,polynomial_equation
842,Determine the zeros of the polynomial function...,polynomial_equation
843,Determine the zeros of the polynomial function...,polynomial_equation


In [2]:
df2 = pd.DataFrame(columns=['question', 'label'])
with open("algebra__polynomial_roots.txt", "r") as data:
    #First 109 instance of Dataset      
    list_item = data.readlines()[:217]
    for item in range(0,len(list_item),2):
        df2 = df2.append({"question":list_item[item], "label":"polynomial_equation"}, ignore_index=True)
    data.close()

initial_dataset = df1.append(df2, ignore_index=True)
print('DATASET PREVIEW')
print(initial_dataset)

DATASET PREVIEW
                                              question                label
0            It is the sum of the terms of a sequence.  series_and_sequence
1    Find the sum of the first ten terms of the ari...  series_and_sequence
2    Find the sum of the first 25 terms of the arit...  series_and_sequence
3    The sum of the first 10 terms of an arithmetic...  series_and_sequence
4    The third term of an arithmetic sequence is −1...  series_and_sequence
..                                                 ...                  ...
949                Factor -35*k**2 + 27415*k - 7830.\n  polynomial_equation
950       Factor -6*n**4/5 - 116*n**3/5 - 38*n**2/5.\n  polynomial_equation
951  What is w in 3*w**3 - 3561*w**2 + 1058505*w - ...  polynomial_equation
952    Find v, given that 12*v**2 - 585*v - 147 = 0.\n  polynomial_equation
953  Determine z so that 3*z**3 + 234*z**2/5 + 864*...  polynomial_equation

[954 rows x 2 columns]


In [3]:
print('DATASET INFORMATION')
print(initial_dataset.label.value_counts())

DATASET INFORMATION
series_and_sequence    318
polynomials            318
polynomial_equation    318
Name: label, dtype: int64


In [4]:
import re

def change_to_dollar(exp):
    exp = re.sub("\.\.\.|\\dots|…","x",exp)
    exp = re.sub("=","$ $",exp)
    return exp

for i in range(len(initial_dataset)):
    qs = initial_dataset.loc[i, 'question']
    fx = re.search('[$].+?[=].+?[$]',qs)
    len_check = len(re.findall('[$].+?[=].+?[$]', qs))
    iplus = 0
    while iplus<len_check:
        iplus += 1
        qs = qs[:fx.span()[0]] + " " + change_to_dollar(fx.group()) + " " + qs[fx.span()[1]:]
        #check for the next LaTex Enclosure if any         
        fx = re.search('[$].+?[=].+?[$]', qs)
    #the new question is added to index i       
    initial_dataset.loc[i,'question'] = qs
    

pattern = r"\\sqrt\{(.+?)\}"
replacement = r"(\1)^{\\frac{1}{2}}"

dataset_sympytolatex = initial_dataset.replace(pattern, replacement, regex=True)

print(dataset_sympytolatex)

                                              question                label
0            It is the sum of the terms of a sequence.  series_and_sequence
1    Find the sum of the first ten terms of the ari...  series_and_sequence
2    Find the sum of the first 25 terms of the arit...  series_and_sequence
3    The sum of the first 10 terms of an arithmetic...  series_and_sequence
4    The third term of an arithmetic sequence is −1...  series_and_sequence
..                                                 ...                  ...
949                Factor -35*k**2 + 27415*k - 7830.\n  polynomial_equation
950       Factor -6*n**4/5 - 116*n**3/5 - 38*n**2/5.\n  polynomial_equation
951  What is w in 3*w**3 - 3561*w**2 + 1058505*w - ...  polynomial_equation
952    Find v, given that 12*v**2 - 585*v - 147 = 0.\n  polynomial_equation
953  Determine z so that 3*z**3 + 234*z**2/5 + 864*...  polynomial_equation

[954 rows x 2 columns]


In [5]:
from latex2sympy2 import latex2sympy
import re

df = dataset_sympytolatex

# Creates a new dataframe for updated question with freeform formula
new_df = pd.DataFrame(columns=['question', 'label'] )

# looping through all data    
for i in range(len(df)):
    new_df.loc[i,'label'] = df.loc[i,'label']
    test = df.loc[i, 'question']
    #regex value for searching all characters from '$' to the next '$' symbol
    check = re.search('[$].+?[$]',test)
    #checks how many instance of latex enclosure are there in the question     
    check_len = len(re.findall('[$].+?[$]',test))
    iteration = 0
    while iteration<check_len:
        iteration += 1
        #concatenate the question string from all characters before formula + formula + all characters after formula
        # formula = re.sub(" ","",str(latex2sympy(r"{}".format(check.group()))))
        # the formula is a converted LaTex to Sympy then removed all white spaces
        print("change problem in index", i)
        print(check.group())
        test = test[:check.span()[0]] + " " + re.sub(" ","",str(latex2sympy(r"{}".format(check.group())))) + " " + test[check.span()[1]:]
        #check for the next LaTex Enclosure if any         
        check = re.search('[$].+?[$]',test)
    #the new question is added to index i       
    new_df.loc[i,'question'] = test

new_df.head()

change problem in index 27
$S_n $
change problem in index 27
$ -80$
change problem in index 27
$a_1 $
change problem in index 27
$ 10$
change problem in index 27
$a_n $
change problem in index 27
$ -26$
change problem in index 28
$S_n  $
change problem in index 28
$ 50$
change problem in index 28
$a_1 $
change problem in index 28
$ 4$
change problem in index 28
$a_n $
change problem in index 28
$ 16$
change problem in index 29
$S_n $
change problem in index 29
$ -15$
change problem in index 29
$a_1 $
change problem in index 29
$ 12$
change problem in index 30
$S_n $
change problem in index 30
$ 180$
change problem in index 30
$a_1 $
change problem in index 30
$ 5$
change problem in index 49
$a_n $
change problem in index 49
$ 5 + 3n$
change problem in index 61
$a_n $
change problem in index 61
$ 5n-1$
change problem in index 62
$a_n+\frac{(-1)^n}{n^2}$
change problem in index 64
$-\frac{1}{2}$
change problem in index 64
$\frac{1}{4}$
change problem in index 64
$-\frac{1}{6}$
change pro

change problem in index 271
$ x-2$
change problem in index 272
$P(x)$
change problem in index 272
$x-r$
change problem in index 273
$ x^3+x^2+x+3$
change problem in index 273
$x $
change problem in index 273
$ -2$
change problem in index 274
$ 3x-2$
change problem in index 275
$ 2x^2-5x+3$
change problem in index 276
$x^2-7x-4$
change problem in index 276
$ x-20$
change problem in index 278
$2x^3-7x^2-19x+20$
change problem in index 278
$x-5$
change problem in index 279
$x-3$
change problem in index 279
$ x^4-3x^3-x+3$
change problem in index 280
$x^3-7x^2+4x-28$
change problem in index 280
$x-7$
change problem in index 281
$f(x)  $
change problem in index 281
$  x^3-5x^2-7x+4$
change problem in index 281
$ f(1)$
change problem in index 282
$f(x) $
change problem in index 282
$ x^3-5x^2-7x+4$
change problem in index 282
$f(-2)$
change problem in index 283
$f(x) $
change problem in index 283
$ x^3-5x^2-7x+4$
change problem in index 283
$f(\frac{1}{2})$
change problem in index 284
$g(x) 

change problem in index 407
$2,1,\frac{1}{2}$
change problem in index 408
$2,1,\frac{1}{2}$
change problem in index 408
$S_{\infty}$
change problem in index 410
$\frac{2}{3}$
change problem in index 411
$4,6,9$
change problem in index 412
$ 45, 15, 5$
change problem in index 413
$2, 8, 32,128$
change problem in index 414
$S_7$
change problem in index 415
$S_6$
change problem in index 416
$S_8$
change problem in index 417
$a_1 $
change problem in index 417
$ 2$
change problem in index 417
$ r $
change problem in index 417
$ -4$
change problem in index 417
$S_8$
change problem in index 418
$a_1 $
change problem in index 418
$ \frac{1}{2}$
change problem in index 418
$r $
change problem in index 418
$ -2$
change problem in index 418
$S_6$
change problem in index 419
$12,6,3$
change problem in index 420
$125,25,5$
change problem in index 421
$1,\frac{1}{5},\frac{1}{25}$
change problem in index 422
$15,5,\frac{5}{3}$
change problem in index 423
$1,\frac{2}{3},\frac{4}{9}$
change problem in 

change problem in index 522
$ 0$
change problem in index 523
$x^4 + 10x + 9 $
change problem in index 523
$ 0$
change problem in index 524
$x^3 + 9x^2 + 22x + 12 $
change problem in index 524
$ 0$
change problem in index 525
$4x^3 - 18x^2 + 10x - 1 $
change problem in index 525
$ 0$
change problem in index 529
$\frac{p}{q}$
change problem in index 530
$\frac{p}{q}$
change problem in index 532
$x $
change problem in index 532
$ 9$
change problem in index 533
$2m + 1$
change problem in index 536
$p(b)$
change problem in index 536
$ 6b^3 - 3b^2 + 2b - 4$
change problem in index 537
$4x^4 + 31x^3 - 4x^2 - 89x + 22 $
change problem in index 537
$ 0$
change problem in index 538
$p(x) $
change problem in index 538
$ 2x^3 - 14x^2 + x - 7$
change problem in index 539
$4x^3 + 4x^2 - 31x + 5 $
change problem in index 539
$ 0 $
change problem in index 540
$x^2 - 3kx + 3k - 7 $
change problem in index 540
$ 0$
change problem in index 541
$4x^4 + 31x^3 - 4x^2 - 89x + 22 $
change problem in index 541

change problem in index 665
$0$
change problem in index 667
$P\left(r\right)$
change problem in index 667
$0$
change problem in index 667
$\left(x-3\right)$
change problem in index 667
$P\left(x\right)$
change problem in index 668
$p\left(x\right)$
change problem in index 668
$3x^3+9x^2+4x+12$
change problem in index 669
$p\left(x\right)$
change problem in index 669
$2x^3+9x^2+19x+15$
change problem in index 669
$-\frac{3}{2}$
change problem in index 670
$x^3-9x^2$
change problem in index 670
$0$
change problem in index 670
$\left(x-9\right)$
change problem in index 671
$34cm^3$
change problem in index 672
$x^3-2x^2+5x-4\div x+1$
change problem in index 673
$x^3+7x^2+5x-10 \div x+5$
change problem in index 674
$4x^3+15x^2+2x-21 \div x+3$
change problem in index 675
$4x^4+22x^3-2x-12x^2+18 \div x+6$
change problem in index 676
$x^5-3x^4-2x^3+6x^2+5x-15 \div x-3$
change problem in index 677
$(x^3-7x^2+2x+4) \div (x+3)$
change problem in index 678
$(x^3-4x^2-5x+6) \div (x+3)$
change probl

change problem in index 785
$0 $
change problem in index 787
$x^3+x^2+\frac{3}{4}x+25$
change problem in index 787
$0 $
change problem in index 788
$x^2 +2x+1$
change problem in index 788
$0 $
change problem in index 789
$5x^3+\frac{5}{x}$
change problem in index 789
$0 $
change problem in index 790
$5x^{-2}+5$
change problem in index 790
$0 $
change problem in index 791
$10^x+2x+1$
change problem in index 791
$0 $
change problem in index 792
$-x^2+10$
change problem in index 792
$0 $
change problem in index 793
$\frac{1}{2}x^2-1$
change problem in index 793
$0  $
change problem in index 794
$\frac{3}{4}x^4y^3-21x$
change problem in index 794
$0 $
change problem in index 795
$9+(x)^{\frac{1}{2}}-3$
change problem in index 795
$0 $
change problem in index 796
$(y)^{\frac{1}{2}}+2$
change problem in index 796
$0 $
change problem in index 797
$\frac{1}{4}x^3+2x$
change problem in index 797
$0 $
change problem in index 798
$2x^2+3x+1$
change problem in index 798
$0 $
change problem in inde

,question,label
0,It is the sum of the terms of a sequence.,series_and_sequence
1,Find the sum of the first ten terms of the ari...,series_and_sequence
2,Find the sum of the first 25 terms of the arit...,series_and_sequence
3,The sum of the first 10 terms of an arithmetic...,series_and_sequence
4,The third term of an arithmetic sequence is −1...,series_and_sequence


In [6]:
new_df = new_df.replace(' \+ ', '+', regex=True)
new_df = new_df.replace(' - ', '-', regex=True)
# We remove the equal sign so that the sympy library would not automatically evaluate the expression
new_df = new_df.replace(' = ', ' ', regex=True)
new_df = new_df.replace('\n','', regex=True) #white space
new_df = new_df.replace('\?','', regex=True) #Question marks
new_df = new_df.replace('\'','', regex=True) #quotation marks
new_df = new_df.replace(',','', regex=True)  #comma
new_df = new_df.replace('[\.]','', regex=True) #periods
new_df = new_df.replace('\t', ' ', regex=True) #tabs
new_df = new_df.replace('[;]', ' ', regex=True) #tabs
new_df = new_df.replace(':', ' ', regex=True) #tabs
new_df = new_df.replace(r"\_\_+", "__", regex=True)
dataset_freeform = new_df

In [7]:
question_list = new_df.drop(columns=['label']).values.tolist()
question_list

[['It is the sum of the terms of a sequence'],
 ['Find the sum of the first ten terms of the arithmetic sequence 41016'],
 ['Find the sum of the first 25 terms of the arithmetic sequence 17 22 2732 …'],
 ['The sum of the first 10 terms of an arithmetic sequence is 530 What is the first term if the last term is 80 and the common difference is 2'],
 ['The third term of an arithmetic sequence is −12 and the seventh term is 8 What is the sum of the first 10 terms'],
 ['Find the sum of the first 50 terms of the arithmetic sequence if the first term is 21 and the twentieth term is 154'],
 ['Find the sum of the first eighteen terms of the arithmetic sequence whose nth term is '],
 ['The first term of an arithmetic sequence is 5 the last term is 45 and the sum is 275 Find the number of terms'],
 ['If the first n terms of the arithmetic sequence 20 18 16 are added how many of these terms will be added to get a sum of -100'],
 ['A yaya receives a starting annual salary of Php 3600000 with a year

In [8]:
from sympy import *
from sympy.parsing.sympy_parser import parse_expr
import re

def tokenize_formula(expression):
    try:
        return srepr(parse_expr(expression, evaluate=False))
    except:
        return expression

for i in range(len(question_list)):
    item_question = question_list[i][0]
    tokenize_question_item = item_question.split()
    for x in range(len(tokenize_question_item)):
        check = re.findall('[\+]|[\*]|[-]|[/]|[0-9]',tokenize_question_item[x])
        if len(tokenize_question_item[x]) == 1:
            check = True 
        if check: 
            print("index: ",i)
            print(tokenize_question_item[x])
            text = re.sub("\s|[,]","",str(tokenize_formula(tokenize_question_item[x])))
            print(text)
            text = re.sub("\d+","(d)",text)
            text = re.sub("[(]['].+?['][)]","(x)",text)
#             text = re.sub("Symbol[(]['].+?['][)]","symbol(x)",text)
            text = re.sub('\s+', '', text)
            tokenize_question_item[x] = text

    question_list[i] = ' '.join(tokenize_question_item)

index:  0
a
Symbol('a')
index:  1
41016
Integer(41016)
index:  2
25
Integer(25)
index:  2
17
Integer(17)
index:  2
22
Integer(22)
index:  2
2732
Integer(2732)
index:  2
…
…
index:  3
10
Integer(10)
index:  3
530
Integer(530)
index:  3
80
Integer(80)
index:  3
2
Integer(2)
index:  4
−12
−12
index:  4
8
Integer(8)
index:  4
10
Integer(10)
index:  5
50
Integer(50)
index:  5
21
Integer(21)
index:  5
154
Integer(154)
index:  7
5
Integer(5)
index:  7
45
Integer(45)
index:  7
275
Integer(275)
index:  8
n
Symbol('n')
index:  8
20
Integer(20)
index:  8
18
Integer(18)
index:  8
16
Integer(16)
index:  8
a
Symbol('a')
index:  8
-100
Integer(-100)
index:  9
A
Symbol('A')
index:  9
a
Symbol('a')
index:  9
3600000
Integer(3600000)
index:  9
a
Symbol('a')
index:  9
300000
Integer(300000)
index:  9
8
Integer(8)
index:  10
a
Symbol('a')
index:  10
1000
Integer(1000)
index:  10
1300
Integer(1300)
index:  10
1600
Integer(1600)
index:  10
52
Integer(52)
index:  11
a
Symbol('a')
index:  11
600000
Integer(60

Add(Mul(Integer(6)Symbol('a'))Integer(1))
index:  113
5
Integer(5)
index:  113
3
Integer(3)
index:  113
1
Integer(1)
index:  113
-1
Integer(-1)
index:  113
-3
Integer(-3)
index:  113
…
…
index:  114
I
I
index:  114
I
I
index:  114
40m
40m
index:  114
I
I
index:  114
60m
60m
index:  114
80m
80m
index:  114
I
I
index:  114
10th
10th
index:  114
I
I
index:  116
04812
04812
index:  117
3
Integer(3)
index:  117
-2
Integer(-2)
index:  117
-7\dots
-7\dots
index:  118
23
Integer(23)
index:  118
18
Integer(18)
index:  118
13
Integer(13)
index:  118
8
Integer(8)
index:  118
3
Integer(3)
index:  118
-7
Integer(-7)
index:  118
-12
Integer(-12)
index:  119
-7
Integer(-7)
index:  119
-4
Integer(-4)
index:  119
-1
Integer(-1)
index:  119
2
Integer(2)
index:  119
5\dots
5\dots
index:  120
3
Integer(3)
index:  120
13/4
Mul(Integer(13)Pow(Integer(4)Integer(-1)))
index:  120
7/2
Mul(Integer(7)Pow(Integer(2)Integer(-1)))
index:  120
15/4
Mul(Integer(15)Pow(Integer(4)Integer(-1)))
index:  122
a_3
Symbol('a

Integer(18)
index:  192
92
Integer(92)
index:  193
-14
Integer(-14)
index:  193
6
Integer(6)
index:  194
24
Integer(24)
index:  194
-8
Integer(-8)
index:  195
6
Integer(6)
index:  195
-18
Integer(-18)
index:  196
2*5**(1/2)
Mul(Integer(2)Pow(Integer(5)Mul(Integer(1)Pow(Integer(2)Integer(-1)))))
index:  196
14*5**(1/2)
Mul(Integer(14)Pow(Integer(5)Mul(Integer(1)Pow(Integer(2)Integer(-1)))))
index:  197
x
Symbol('x')
index:  197
9*x
Mul(Integer(9)Symbol('x'))
index:  198
31
Integer(31)
index:  198
85
Integer(85)
index:  199
17
Integer(17)
index:  199
23
Integer(23)
index:  199
29
Integer(29)
index:  200
6
Integer(6)
index:  200
30
Integer(30)
index:  201
5
Integer(5)
index:  201
9
Integer(9)
index:  201
13
Integer(13)
index:  201
17
Integer(17)
index:  201
…
…
index:  201
409
Integer(409)
index:  202
a_1
Symbol('a_1')
index:  202
5
Integer(5)
index:  202
d
Symbol('d')
index:  202
5
Integer(5)
index:  202
n
Symbol('n')
index:  202
25
Integer(25)
index:  203
5
Integer(5)
index:  203
9
Inte

2*x**3-1*7*x**2-1*19*x+20
Add(Mul(Integer(2)Pow(Symbol('x')Integer(3)))Mul(Integer(-1)Mul(Integer(1)Integer(7)Pow(Symbol('x')Integer(2))))Mul(Integer(-1)Mul(Integer(1)Integer(19)Symbol('x')))Integer(20))
index:  278
x-5
Add(Symbol('x')Mul(Integer(-1)Integer(5)))
index:  279
x-3
Add(Symbol('x')Mul(Integer(-1)Integer(3)))
index:  279
a
Symbol('a')
index:  279
x**4-1*3*x**3-x+3
Add(Pow(Symbol('x')Integer(4))Mul(Integer(-1)Mul(Integer(1)Integer(3)Pow(Symbol('x')Integer(3))))Mul(Integer(-1)Symbol('x'))Integer(3))
index:  280
x**3-1*7*x**2+4*x-28
Add(Pow(Symbol('x')Integer(3))Mul(Integer(-1)Mul(Integer(1)Integer(7)Pow(Symbol('x')Integer(2))))Mul(Integer(4)Symbol('x'))Mul(Integer(-1)Integer(28)))
index:  280
x-7
Add(Symbol('x')Mul(Integer(-1)Integer(7)))
index:  280
a
Symbol('a')
index:  281
x**3-1*5*x**2-1*7*x+4
Add(Pow(Symbol('x')Integer(3))Mul(Integer(-1)Mul(Integer(1)Integer(5)Pow(Symbol('x')Integer(2))))Mul(Integer(-1)Mul(Integer(1)Integer(7)Symbol('x')))Integer(4))
index:  281
f(1)
Func

\
index:  330
2*x**4+3*x**2+4*x+1
Add(Mul(Integer(2)Pow(Symbol('x')Integer(4)))Mul(Integer(3)Pow(Symbol('x')Integer(2)))Mul(Integer(4)Symbol('x'))Integer(1))
index:  330
\
\
index:  331
2*x**4+3*x**2+4*x+1
Add(Mul(Integer(2)Pow(Symbol('x')Integer(4)))Mul(Integer(3)Pow(Symbol('x')Integer(2)))Mul(Integer(4)Symbol('x'))Integer(1))
index:  331
\
\
index:  332
2*x**4+3*x**2+4*x+1
Add(Mul(Integer(2)Pow(Symbol('x')Integer(4)))Mul(Integer(3)Pow(Symbol('x')Integer(2)))Mul(Integer(4)Symbol('x'))Integer(1))
index:  332
\
\
index:  333
5*x**3
Mul(Integer(5)Pow(Symbol('x')Integer(3)))
index:  334
5*x**3
Mul(Integer(5)Pow(Symbol('x')Integer(3)))
index:  335
5*x**3
Mul(Integer(5)Pow(Symbol('x')Integer(3)))
index:  336
5*x**3
Mul(Integer(5)Pow(Symbol('x')Integer(3)))
index:  337
x**6-1*3*x**3+1
Add(Pow(Symbol('x')Integer(6))Mul(Integer(-1)Mul(Integer(1)Integer(3)Pow(Symbol('x')Integer(3))))Integer(1))
index:  338
x**6-1*3*x**3+1
Add(Pow(Symbol('x')Integer(6))Mul(Integer(-1)Mul(Integer(1)Integer(3)Pow(

...

Add(Symbol('x')Integer(12))
index:  743
a
Symbol('a')
index:  743
3*x**3+6*x+9
Add(Mul(Integer(3)Pow(Symbol('x')Integer(3)))Mul(Integer(6)Symbol('x'))Integer(9))
index:  743
x+3
Add(Symbol('x')Integer(3))
index:  744
a
Symbol('a')
index:  744
x**3+12*x**2+5*x-26
Add(Pow(Symbol('x')Integer(3))Mul(Integer(12)Pow(Symbol('x')Integer(2)))Mul(Integer(5)Symbol('x'))Mul(Integer(-1)Integer(26)))
index:  744
x+3
Add(Symbol('x')Integer(3))
index:  745
a
Symbol('a')
index:  745
4*x**3+7*x**2+3*x+5
Add(Mul(Integer(4)Pow(Symbol('x')Integer(3)))Mul(Integer(7)Pow(Symbol('x')Integer(2)))Mul(Integer(3)Symbol('x'))Integer(5))
index:  745
x+5
Add(Symbol('x')Integer(5))
index:  746
a
Symbol('a')
index:  746
2*y**3-1*3*y**2-1*13*y+5
Add(Mul(Integer(2)Pow(Symbol('y')Integer(3)))Mul(Integer(-1)Mul(Integer(1)Integer(3)Pow(Symbol('y')Integer(2))))Mul(Integer(-1)Mul(Integer(1)Integer(13)Symbol('y')))Integer(5))
index:  746
y-3
Add(Symbol('y')Mul(Integer(-1)Integer(3)))
index:  747
a
Symbol('a')
index:  747
4*x**

index:  800
1*x**3/4
Mul(Integer(1)Pow(Integer(4)Integer(-1))Pow(Symbol('x')Integer(3)))
index:  800
0
Integer(0)
index:  801
a
Symbol('a')
index:  801
-3+1/x
Add(Integer(-3)Mul(Integer(1)Pow(Symbol('x')Integer(-1))))
index:  801
0
Integer(0)
index:  802
a
Symbol('a')
index:  802
x**20-1
Add(Pow(Symbol('x')Integer(20))Mul(Integer(-1)Integer(1)))
index:  802
0
Integer(0)
index:  803
4
Integer(4)
index:  803
240*m**2
Mul(Integer(240)Pow(Symbol('m')Integer(2)))
index:  804
4
Integer(4)
index:  804
240*m**2
Mul(Integer(240)Pow(Symbol('m')Integer(2)))
index:  805
4
Integer(4)
index:  805
240*m**2
Mul(Integer(240)Pow(Symbol('m')Integer(2)))
index:  806
4
Integer(4)
index:  806
240*m**2
Mul(Integer(240)Pow(Symbol('m')Integer(2)))
index:  807
1*x**2/2-x+2
Add(Mul(Integer(1)Pow(Integer(2)Integer(-1))Pow(Symbol('x')Integer(2)))Mul(Integer(-1)Symbol('x'))Integer(2))
index:  807
0
Integer(0)
index:  808
3*x**4+5*x**3+3*x**2-x+1
Add(Mul(Integer(3)Pow(Symbol('x')Integer(4)))Mul(Integer(5)Pow(Symbol(

index:  866
-2*u**2/3+74*u/3-140
Add(Mul(Integer(-1)Integer(2)Pow(Integer(3)Integer(-1))Pow(Symbol('u')Integer(2)))Mul(Integer(74)Pow(Integer(3)Integer(-1))Symbol('u'))Mul(Integer(-1)Integer(140)))
index:  866
0
Integer(0)
index:  866
u
Symbol('u')
index:  867
f
Symbol('f')
index:  867
-2*f**4-56*f**3-288*f**2-544*f-352
Add(Mul(Integer(-1)Integer(2)Pow(Symbol('f')Integer(4)))Mul(Integer(-1)Mul(Integer(56)Pow(Symbol('f')Integer(3))))Mul(Integer(-1)Mul(Integer(288)Pow(Symbol('f')Integer(2))))Mul(Integer(-1)Mul(Integer(544)Symbol('f')))Mul(Integer(-1)Integer(352)))
index:  867
0
Integer(0)
index:  868
3*u**3/4+3*u**2-9*u/4-27/2
Add(Mul(Integer(3)Pow(Integer(4)Integer(-1))Pow(Symbol('u')Integer(3)))Mul(Integer(3)Pow(Symbol('u')Integer(2)))Mul(Integer(-1)Mul(Integer(9)Pow(Integer(4)Integer(-1))Symbol('u')))Mul(Integer(-1)Mul(Integer(27)Pow(Integer(2)Integer(-1)))))
index:  869
t
Symbol('t')
index:  869
-2*t**2/7-3246*t/7-3244/7
Add(Mul(Integer(-1)Integer(2)Pow(Integer(7)Integer(-1))Pow(Symb

Integer(0)
index:  901
s**2/2-17*s+93/2
Add(Mul(Pow(Integer(2)Integer(-1))Pow(Symbol('s')Integer(2)))Mul(Integer(-1)Mul(Integer(17)Symbol('s')))Mul(Integer(93)Pow(Integer(2)Integer(-1))))
index:  902
-u**2/5-5878*u/5-8637721/5
Add(Mul(Integer(-1)Mul(Integer(5878)Pow(Integer(5)Integer(-1))Symbol('u')))Mul(Pow(Integer(5)Integer(-1))Mul(Integer(-1)Pow(Symbol('u')Integer(2))))Mul(Integer(-1)Mul(Integer(8637721)Pow(Integer(5)Integer(-1)))))
index:  903
l
Symbol('l')
index:  903
-40*l**5+874*l**4+2056*l**3+200*l**2-192*l
Add(Mul(Integer(-1)Integer(40)Pow(Symbol('l')Integer(5)))Mul(Integer(874)Pow(Symbol('l')Integer(4)))Mul(Integer(2056)Pow(Symbol('l')Integer(3)))Mul(Integer(200)Pow(Symbol('l')Integer(2)))Mul(Integer(-1)Mul(Integer(192)Symbol('l'))))
index:  903
0
Integer(0)
index:  904
r**4+7*r**3-57*r**2-455*r-392
Add(Pow(Symbol('r')Integer(4))Mul(Integer(7)Pow(Symbol('r')Integer(3)))Mul(Integer(-1)Mul(Integer(57)Pow(Symbol('r')Integer(2))))Mul(Integer(-1)Mul(Integer(455)Symbol('r')))Mul(In

index:  941
-v**4+13*v**3+118*v**2-52*v-456
Add(Mul(Integer(-1)Pow(Symbol('v')Integer(4)))Mul(Integer(13)Pow(Symbol('v')Integer(3)))Mul(Integer(118)Pow(Symbol('v')Integer(2)))Mul(Integer(-1)Mul(Integer(52)Symbol('v')))Mul(Integer(-1)Integer(456)))
index:  941
0
Integer(0)
index:  941
v
Symbol('v')
index:  942
-3*k**3+354*k**2-1380*k+1368
Add(Mul(Integer(-1)Integer(3)Pow(Symbol('k')Integer(3)))Mul(Integer(354)Pow(Symbol('k')Integer(2)))Mul(Integer(-1)Mul(Integer(1380)Symbol('k')))Integer(1368))
index:  943
-x**2/5-38*x/5+43
Add(Mul(Integer(-1)Mul(Integer(38)Pow(Integer(5)Integer(-1))Symbol('x')))Mul(Pow(Integer(5)Integer(-1))Mul(Integer(-1)Pow(Symbol('x')Integer(2))))Integer(43))
index:  943
0
Integer(0)
index:  943
x
Symbol('x')
index:  944
-2*q**2/13-452*q/13-450/13
Add(Mul(Integer(-1)Integer(2)Pow(Integer(13)Integer(-1))Pow(Symbol('q')Integer(2)))Mul(Integer(-1)Mul(Integer(452)Pow(Integer(13)Integer(-1))Symbol('q')))Mul(Integer(-1)Mul(Integer(450)Pow(Integer(13)Integer(-1)))))
index:

In [9]:
# Adding the correlated first item in pair for the correlated column
remove_words = pd.read_csv('remove_words_v1.csv')
words = remove_words['remove_words'].values.tolist()
words

['(l)',
 '(polynomial)',
 '(w)',
 'add(mul(integer((d))pow(symbol(x)integer((d))))symbol(x)mul(integer(-(d))mul(integer((d))integer((d))pow(mul(integer((d))symbol(x))mul(integer((d))pow(integer((d))integer(-(d))))))))',
 'add(mul(integer(-(d))integer((d))pow(integer((d))mul(integer((d))pow(integer((d))integer(-(d))))))integer((d)))',
 'add(symbol(x)mul(integer(-(d))mul(integer((d))integer((d))symbol(x))))',
 'add(symbol(x)symbol(x))',
 'advised',
 'allowance',
 'answered',
 'any)',
 'asked',
 'bank',
 'blank',
 'box',
 'buy',
 'cardboard',
 'centimeter',
 'centimeters',
 'chain',
 'christmas',
 'classmate',
 'classroom',
 'closely',
 'cm',
 'composed',
 'consider',
 'constant',
 'construction',
 'contains',
 'continue',
 'continued',
 'corner',
 'correct',
 'costs',
 'crew',
 'cruz',
 'cubes',
 'daily',
 'december',
 'decimal',
 'deposits',
 'describes',
 'description',
 'dimension',
 'dimensions',
 'dimesnions',
 'dish',
 'distance',
 'div',
 'division',
 'doubled',
 'doubles',
 'drop

In [10]:
import sklearn
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

words += stopwords.words('english')

vect = CountVectorizer(token_pattern = '[a-zA-Z0-9,=+-<>()_{}]+',stop_words = words, min_df=0)
vect.fit(question_list)

CountVectorizer(min_df=0,
                stop_words=['(l)', '(polynomial)', '(w)',
                            'add(mul(integer((d))pow(symbol(x)integer((d))))symbol(x)mul(integer(-(d))mul(integer((d))integer((d))pow(mul(integer((d))symbol(x))mul(integer((d))pow(integer((d))integer(-(d))))))))',
                            'add(mul(integer(-(d))integer((d))pow(integer((d))mul(integer((d))pow(integer((d))integer(-(d))))))in...
                            'add(symbol(x)mul(integer(-(d))mul(integer((d))integer((d))symbol(x))))',
                            'add(symbol(x)symbol(x))', 'advised', 'allowance',
                            'answered', 'any)', 'asked', 'bank', 'blank', 'box',
                            'buy', 'cardboard', 'centimeter', 'centimeters',
                            'chain', 'christmas', 'classmate', 'classroom',
                            'closely', 'cm', 'composed', 'consider', 'constant',
                            'construction', 'contains', ...],
           

In [11]:
print("Column Names and their count")
print(format(vect.vocabulary_))

Column Names and their count
{'sum': 559, 'symbol(x)': 562, 'sequence': 542, 'find': 364, 'first': 366, 'ten': 563, 'arithmetic': 288, 'integer((d))': 394, 'term': 565, 'last': 397, 'common': 311, 'difference': 333, 'third': 568, '(d)': 1, 'seventh': 545, 'twentieth': 573, 'eighteen': 343, 'whose': 586, 'nth': 500, 'number': 501, 'added': 274, 'many': 408, 'get': 378, 'integer(-(d))': 395, 'starting': 555, 'annual': 281, 'php': 515, 'pair': 510, 'able': 19, 'week': 582, 'second': 541, 'pattern': 511, 'made': 406, 'much': 414, 'annually': 282, 'three': 570, 'day': 325, 'end': 346, 'de': 326, 'business': 304, 'since': 548, 'one': 505, 'additional': 275, 'natural': 496, 'numbers': 502, 'give': 379, 'integers': 396, 'divisible': 338, '(d)+(d)+(d)+': 2, '-(d)+(-(d))+(d)+': 14, 's_n': 540, 'a_n': 17, '(d)n': 5, 'even': 351, 'suppose': 561, 'cinema': 308, 'two': 574, 'add(integer((d))mul(integer(-(d))symbol(x)))': 28, 'continues': 317, 'following': 368, 'formula': 370, 'eighth': 344, 'tenth':

In [12]:
print("Total Columns")
print(len(vect.get_feature_names_out()))

Total Columns
591


In [13]:
bag_of_words = vect.transform(question_list)
format(bag_of_words.toarray())

'[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n ...\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]]'

In [14]:
import scipy.sparse

bow_Matrix = pd.DataFrame.sparse.from_spmatrix(bag_of_words, columns = vect.get_feature_names_out())
X = bow_Matrix.values
y = new_df.loc[:,"label"].values
bow_Matrix.head()

,(binomial),(d),(d)+(d)+(d)+,(d)cm,(d)m,(d)n,(d)nd,(d)rd,(d)s_(d),(d)st,...,way,week,whatever,whati,whether,whose,within,would,write,x
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state= 0)

In [29]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100,random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [30]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[77  3  2]
 [ 7 78  0]
 [ 2  0 70]]
                     precision    recall  f1-score   support

polynomial_equation       0.90      0.94      0.92        82
        polynomials       0.96      0.92      0.94        85
series_and_sequence       0.97      0.97      0.97        72

           accuracy                           0.94       239
          macro avg       0.94      0.94      0.94       239
       weighted avg       0.94      0.94      0.94       239



In [18]:
# import pickle
# ## Uncomment to produce the file
# ## save the model to disk
# model = 'rfa_model.sav'
# pickle.dump(classifier, open(model, 'wb'))
# myvectorizer = 'vectorizer.sav'
# pickle.dump(vect, open(myvectorizer, 'wb'))

In [27]:
accuracy_score(y_test, y_pred)

0.9414225941422594